# Import package dan Import file


In [1]:
import numpy as np
import pandas as pd
import math
from tqdm import tqdm

# LOAD DATA
file_name = "Data Stock 9 Oct 2024.xlsx"
file_path = f"../Data/{file_name}"
master_path = "../Data/MASTER REGION STO_NEXT VERSION (62).xlsx"
df_origin = pd.read_excel(file_path, sheet_name="dos by store-brand type & area")
df_master = pd.read_excel(master_path, sheet_name="REGION 3", header=1)

# mengambil kolo 'SITE CODE' dan 'PT' dari file master
df_master_selected = df_master[['SITE CODE', 'PT']]
# lookup PT sesuai dengan SITE CODE
df = pd.merge(df_origin, df_master_selected, on='SITE CODE', how='left')
# membuat kolom baru bernama 'store' yang berisi 'store name (site code)'
df['STORE'] = df['STORE NAME'] + " (" + df['SITE CODE'].astype(str) + ")"
# menghapus kolom store name dan site code
df = df.drop(columns=['STORE NAME', 'SITE CODE'])
# mengganti bilangan negatif dan na di sales dan dos dengan 0
df[['Sales 30 days', 'DOS 30 days']] = df[['Sales 30 days','DOS 30 days']].clip(lower=0).fillna(0)

print(df)

               AREA                KOTA          TSH Brand Name  \
0      JAWA BARAT 1       KAB. KARAWANG  DENY MURPHY      APPLE   
1      JAWA BARAT 1       KAB. KARAWANG  DENY MURPHY      APPLE   
2      JAWA BARAT 1       KAB. KARAWANG  DENY MURPHY       ASUS   
3      JAWA BARAT 1       KAB. KARAWANG  DENY MURPHY    Infinix   
4      JAWA BARAT 1       KAB. KARAWANG  DENY MURPHY    Infinix   
...             ...                 ...          ...        ...   
43436  KALIMANTAN 5  KOTA PALANGKA RAYA        BADRI    SAMSUNG   
43437  KALIMANTAN 2    KOTA BANJARMASIN    ADE FAJRI    SAMSUNG   
43438  KALIMANTAN 2    KOTA BANJARMASIN    ADE FAJRI    SAMSUNG   
43439  KALIMANTAN 2    KOTA BANJARMASIN    ADE FAJRI    SAMSUNG   
43440  KALIMANTAN 2    KOTA BANJARMASIN    ADE FAJRI    SAMSUNG   

           Material Status Article code no color type only  \
0              AKTIF STOCK                       Iphone 11   
1              AKTIF STOCK                  iPhone 15 Plus   
2        

# Seleksi PT, KOTA, dan ARTICLE Sesuai yang dicari

In [2]:
pt = 'EAR'
df_pt = df[df['PT'] == pt]

jumlah_baris_kosong = df_pt['Article code no color'].isnull().sum()
print(f"\n{pt} : Jumlah baris yang kolom 'Article'-nya NaN: {jumlah_baris_kosong}")
df_pt = df_pt.dropna(subset=['Article code no color'])
print(df_pt.shape)

category_col_name = 'KOTA'
category = 'KAB. KARAWANG'
df_category = df_pt[df_pt[f'{category_col_name}'] == category]

article = 'OPPO A3X 4/64GB'
df_article = df_category[df_category['Article code no color'] == article]



EAR : Jumlah baris yang kolom 'Article'-nya NaN: 8
(39417, 18)


# Memisahkan store asal (dos berlebih) dengan store tujuan (dos kurang)

In [3]:
def split_store(df, category_col_name):
    df_asal = df[
        ((df['DOS 30 days'] >= 30) & (df['Stock'] >= 2)) |
        ((df['Sales 30 days'] >= 0) & (df['Stock'] >= 2))
    ]
    df_asal = df_asal[[f"{category_col_name}", "STORE", "Article code no color", "Stock", "Sales 30 days", "DOS 30 days"]]
    
    df_tujuan = df[
        (df['DOS 30 days'] <= 23) &
        (df['Sales 30 days'] != 0)
    ]
    df_tujuan = df_tujuan[[f"{category_col_name}", "STORE", "Article code no color", "Stock", "Sales 30 days", "DOS 30 days"]]
    
    return df_asal, df_tujuan

In [4]:

df_asal, df_tujuan = split_store(df_article, category_col_name)
print(df_asal)
print('--------------------------------------------------------')
print(df_tujuan)

                KOTA                                   STORE  \
16737  KAB. KARAWANG               LOTTEMART KARAWANG (G330)   
18465  KAB. KARAWANG   ERAFONE KARAWANG CENTRAL PLAZA (E054)   
18466  KAB. KARAWANG    ERAFONE RESINDA PARK KARAWANG (E123)   
18467  KAB. KARAWANG    ERAFONE RUKO KOSAMBI KARAWANG (E261)   
18468  KAB. KARAWANG     ERAFONE RUKO JUANDA CIKAMPEK (E310)   
18469  KAB. KARAWANG      ERAFONE RUKO RENGASDENGKLOK (E409)   
18470  KAB. KARAWANG          ERAFONE RUKO PINAYUNGAN (E641)   
18471  KAB. KARAWANG               ERAFONE TELUKJAMBE (E781)   
18472  KAB. KARAWANG   ERAFONE RUKO SUROTOKUNTO KLARI (E940)   
18473  KAB. KARAWANG    ERAFONE RUKO TUPAREV KARAWANG (E972)   
18476  KAB. KARAWANG   MEGASTORE RUKO A YANI CIKAMPEK (M064)   
18477  KAB. KARAWANG  MEGASTORE RUKO A. YANI KARAWANG (M102)   

      Article code no color  Stock  Sales 30 days  DOS 30 days  
16737       OPPO A3X 4/64GB      3            1.0         90.0  
18465       OPPO A3X 4/64GB      4   

# Membuat log rotasi 

In [5]:
def custom_round(n):
    return math.ceil(n) if n % 1 >= 0.5 else math.floor(n)

def calculate_dos(stock, sales):
    return custom_round((stock / sales * 30)) if sales != 0 else 0

def rotate_stock(df_asal, df_tujuan, category_col_name):
    log_rotasi = pd.DataFrame(columns=[f"{category_col_name}", "STORE ASAL", "ARTICLE", "BARANG TEROTASI", "STORE TUJUAN"])
    i = 0
    # store asal dan tujuan tidak boleh kosong
    while True:
        dos_df_asal = list(df_asal['DOS 30 days'])
        dos_df_tujuan = list(df_tujuan['DOS 30 days'])
        
        # Pilih store asal dengan melihat dos simulasi terbesar
        df_asal = df_asal.sort_values(by=['DOS 30 days','Stock'], ascending=[False, False]).reset_index(drop=True)
        
        df_asal['Simulated Sales 30 days'] = df_asal['Sales 30 days'].where(df_asal['Sales 30 days'] != 0, 1)
        simulated_doses = []
        for index, row in df_asal.iterrows():
            simulated_stock = row['Stock'] - 1
            simulated_dos = calculate_dos(simulated_stock, row['Simulated Sales 30 days'])
            simulated_doses.append(simulated_dos)
        df_asal.drop('Simulated Sales 30 days', axis=1, inplace=True)
        
        selected_index = simulated_doses.index(max(simulated_doses))
        asal = df_asal.loc[selected_index]

        # buat kondisi dimana rotasi stok di hentikan
        if (simulated_doses[selected_index] < 23) or all(x >= 23 for x in dos_df_tujuan):
            # dos asal yang disimulasikan akan di rotasi kurang dari 23, stop
            # semua store tujuan sudah memiliki dos lebih dari 23, stop
            break

        # Pilih store tujuan dengan DOS terendah yang memiliki sales tertinggi
        df_tujuan = df_tujuan.sort_values(by=['DOS 30 days', 'Sales 30 days'], ascending=[True, False]).reset_index(drop=True)
        tujuan = df_tujuan.loc[0]

        # Update stok dan hitung ulang DOS untuk store asal dan tujuan
        df_asal.at[selected_index, 'Stock'] -= 1
        df_tujuan.at[0, 'Stock'] += 1
        df_asal.at[selected_index, 'DOS 30 days'] = calculate_dos(df_asal.at[selected_index, 'Stock'], df_asal.at[selected_index, 'Sales 30 days'])
        df_tujuan.at[0, 'DOS 30 days'] = calculate_dos(df_tujuan.at[0, 'Stock'], df_tujuan.at[0, 'Sales 30 days'])

        new_log_entry = pd.DataFrame({
            f"{category_col_name}": [asal[f'{category_col_name}']],
            "STORE ASAL": [f"{asal['STORE']}"],
            "ARTICLE": [asal['Article code no color']],
            "BARANG TEROTASI": [1],
            "STORE TUJUAN": [f"{tujuan['STORE']}"]
        })
        
        log_rotasi = pd.concat([log_rotasi, new_log_entry], ignore_index=True)
        
        i += 1
        # print(f'{i}.',asal['STORE'], 'ke' ,tujuan['STORE'])
    
    return df_asal, df_tujuan, log_rotasi

In [6]:
df_asal_updated, df_tujuan_updated, log_rotasi = rotate_stock(df_asal, df_tujuan, category_col_name)
print('\n--------------------------------------------------------')
print('store asal sebelum dirotasi :')
print(df_asal)
print('\n--------------------------------------------------------')
print('store tujuan sebelum dirotasi :')
print(df_tujuan)
print('\n--------------------------------------------------------')
print('log rotasi :')
print(log_rotasi)
print('\n--------------------------------------------------------')
print('store asal setelah dirotasi :')
print(df_asal_updated)
print('\n--------------------------------------------------------')
print('store tujuan setelah dirotasi :')
print(df_tujuan_updated)


--------------------------------------------------------
store asal sebelum dirotasi :
                KOTA                                   STORE  \
16737  KAB. KARAWANG               LOTTEMART KARAWANG (G330)   
18465  KAB. KARAWANG   ERAFONE KARAWANG CENTRAL PLAZA (E054)   
18466  KAB. KARAWANG    ERAFONE RESINDA PARK KARAWANG (E123)   
18467  KAB. KARAWANG    ERAFONE RUKO KOSAMBI KARAWANG (E261)   
18468  KAB. KARAWANG     ERAFONE RUKO JUANDA CIKAMPEK (E310)   
18469  KAB. KARAWANG      ERAFONE RUKO RENGASDENGKLOK (E409)   
18470  KAB. KARAWANG          ERAFONE RUKO PINAYUNGAN (E641)   
18471  KAB. KARAWANG               ERAFONE TELUKJAMBE (E781)   
18472  KAB. KARAWANG   ERAFONE RUKO SUROTOKUNTO KLARI (E940)   
18473  KAB. KARAWANG    ERAFONE RUKO TUPAREV KARAWANG (E972)   
18476  KAB. KARAWANG   MEGASTORE RUKO A YANI CIKAMPEK (M064)   
18477  KAB. KARAWANG  MEGASTORE RUKO A. YANI KARAWANG (M102)   

      Article code no color  Stock  Sales 30 days  DOS 30 days  
16737       OP

# Memisahkan log rotasi untuk melihat total terotasi dan total yang dibutuhakan

In [7]:

def split_log_rotasi(log_rotasi, category_col_name):
    # Membuat tabel store asal beserta stok yang bisa dirotasikan
    tabel_asal = log_rotasi.groupby([f"{category_col_name}", "ARTICLE", "STORE ASAL"])['BARANG TEROTASI'].sum().reset_index()
    tabel_asal.rename(columns={'BARANG TEROTASI': 'TOTAL TEROTASI'}, inplace=True)

    # Membuat tabel store tujuan beserta stok yang dibutuhkan
    tabel_tujuan = log_rotasi.groupby([f"{category_col_name}", "ARTICLE", "STORE TUJUAN"])['BARANG TEROTASI'].sum().reset_index()
    tabel_tujuan.rename(columns={'BARANG TEROTASI': 'TOTAL BUTUH'}, inplace=True)

    return tabel_asal, tabel_tujuan

In [8]:
tabel_asal, tabel_tujuan = split_log_rotasi(log_rotasi, category_col_name)
print('\n--------------------------------------------------------')
print('log rotasi :')
print(log_rotasi)
print('\n--------------------------------------------------------')
print('store yang merotasikan barang :')
print(tabel_asal)
print('\n--------------------------------------------------------')
print('store yang membutuhkan barang :')
print(tabel_tujuan)


--------------------------------------------------------
log rotasi :
             KOTA                              STORE ASAL          ARTICLE  \
0   KAB. KARAWANG               LOTTEMART KARAWANG (G330)  OPPO A3X 4/64GB   
1   KAB. KARAWANG    ERAFONE RESINDA PARK KARAWANG (E123)  OPPO A3X 4/64GB   
2   KAB. KARAWANG               LOTTEMART KARAWANG (G330)  OPPO A3X 4/64GB   
3   KAB. KARAWANG          ERAFONE RUKO PINAYUNGAN (E641)  OPPO A3X 4/64GB   
4   KAB. KARAWANG   MEGASTORE RUKO A YANI CIKAMPEK (M064)  OPPO A3X 4/64GB   
5   KAB. KARAWANG    ERAFONE RESINDA PARK KARAWANG (E123)  OPPO A3X 4/64GB   
6   KAB. KARAWANG  MEGASTORE RUKO A. YANI KARAWANG (M102)  OPPO A3X 4/64GB   
7   KAB. KARAWANG   MEGASTORE RUKO A YANI CIKAMPEK (M064)  OPPO A3X 4/64GB   
8   KAB. KARAWANG  MEGASTORE RUKO A. YANI KARAWANG (M102)  OPPO A3X 4/64GB   
9   KAB. KARAWANG      ERAFONE RUKO RENGASDENGKLOK (E409)  OPPO A3X 4/64GB   
10  KAB. KARAWANG      ERAFONE RUKO RENGASDENGKLOK (E409)  OPPO A3X 4/6

# Memasangkan store asal dan tujuan berdasarkan banyaknya barang (optimal) 

In [9]:
def reorder(log_rotasi):
    df_sorted = log_rotasi.sort_values(by=['TOTAL ASAL'], ascending=[False]).reset_index(drop=True)

    df_temp = pd.DataFrame(df_sorted.iloc[0]).transpose()
    df_sorted.drop(0, inplace=True)

    threshold = 0
    while not df_sorted.empty:
        df_sorted.reset_index(drop=True, inplace=True)
        last_name = df_temp['STORE TUJUAN'].iloc[-1] if not df_temp.empty else None

        for index_sorted, row_sorted in df_sorted.iterrows():
            if last_name == row_sorted['STORE TUJUAN']:
                df_temp = pd.concat([df_temp, pd.DataFrame([row_sorted])], ignore_index=True)
                df_sorted.drop(index_sorted, inplace=True)
                break

        new_threshold = len(df_temp)
        if threshold == new_threshold:
            new_row = df_sorted.iloc[[0]] 
            df_temp = pd.concat([df_temp, new_row], ignore_index=True)
            df_sorted.drop(0, inplace=True)
            df_sorted.reset_index(drop=True, inplace=True)
        threshold = new_threshold

    return df_temp

def process_pairing_1(tabel_asal, tabel_tujuan, paired, category_col_name):
    for index_asal, row_asal in tabel_asal.iterrows():
        for index_tujuan, row_tujuan in tabel_tujuan.iterrows():
            if row_asal['TOTAL TEROTASI'] == row_tujuan['TOTAL BUTUH']:
                new_entry = pd.DataFrame({
                    f"{category_col_name}": [row_asal[f"{category_col_name}"]],
                    "STORE ASAL": [row_asal['STORE ASAL']],
                    "ARTICLE": [row_asal['ARTICLE']],
                    "TOTAL ASAL": [row_asal['TOTAL TEROTASI']],
                    "BARANG TEROTASI": [min(row_asal['TOTAL TEROTASI'], row_tujuan['TOTAL BUTUH'])],
                    "TOTAL TUJUAN": [row_tujuan['TOTAL BUTUH']],
                    "STORE TUJUAN": [row_tujuan['STORE TUJUAN']]
                })
                paired = pd.concat([paired, new_entry], ignore_index=True)
                tabel_asal.drop(index_asal, inplace=True)
                tabel_tujuan.drop(index_tujuan, inplace=True)
                
                return tabel_asal, tabel_tujuan, paired 
    return tabel_asal, tabel_tujuan, paired

def process_pairing_2(tabel_asal, tabel_tujuan, paired, category_col_name):
    tabel_asal['Simulated TOTAL TEROTASI'] = tabel_asal['TOTAL TEROTASI']
    tabel_tujuan['Simulated TOTAL BUTUH'] = tabel_tujuan['TOTAL BUTUH']

    paired_temp = pd.DataFrame(columns=[f"{category_col_name}", 'STORE ASAL', 'ARTICLE', 'TOTAL ASAL', 'BARANG TEROTASI', 'TOTAL TUJUAN', 'STORE TUJUAN'])

    while True:
        tabel_asal = tabel_asal.sort_values(by=['Simulated TOTAL TEROTASI'], ascending=[False]).reset_index(drop=True)
        tabel_tujuan = tabel_tujuan.sort_values(by=['Simulated TOTAL BUTUH'], ascending=[False]).reset_index(drop=True)

        new_entry = pd.DataFrame({
            f"{category_col_name}" : [tabel_asal.loc[0, f"{category_col_name}"]],
            'STORE ASAL' : [tabel_asal.loc[0, 'STORE ASAL']],
            'ARTICLE' : [tabel_asal.loc[0, 'ARTICLE']],
            'TOTAL ASAL' : [tabel_asal.loc[0, 'TOTAL TEROTASI']],
            'BARANG TEROTASI' : min(tabel_asal.loc[0, 'Simulated TOTAL TEROTASI'], tabel_tujuan.loc[0, 'Simulated TOTAL BUTUH']),
            'TOTAL TUJUAN' : [tabel_tujuan.loc[0, 'TOTAL BUTUH']],
            'STORE TUJUAN' : [tabel_tujuan.loc[0, 'STORE TUJUAN']]
        })
        
        paired_temp = pd.concat([paired_temp, new_entry], ignore_index=True)

        tabel_asal.loc[0, 'Simulated TOTAL TEROTASI'] -= new_entry['BARANG TEROTASI'].item()
        tabel_tujuan.loc[0, 'Simulated TOTAL BUTUH'] -= new_entry['BARANG TEROTASI'].item()
            
        if all(x == 0 for x in tabel_tujuan['Simulated TOTAL BUTUH']) : 
            tabel_asal = pd.DataFrame(columns=tabel_asal.columns)
            tabel_tujuan = pd.DataFrame(columns=tabel_tujuan.columns)
            break        

    paired_temp = reorder(paired_temp)
    paired = pd.concat([paired, paired_temp], ignore_index=True)
    
    return tabel_asal, tabel_tujuan, paired

In [10]:
paired = pd.DataFrame(columns=[f"{category_col_name}", 'STORE ASAL', 'ARTICLE', 'TOTAL ASAL', 'BARANG TEROTASI', 'TOTAL TUJUAN', 'STORE TUJUAN'])
threshold = 0
while not tabel_asal.empty and not tabel_tujuan.empty:
    tabel_asal, tabel_tujuan, paired = process_pairing_1(tabel_asal, tabel_tujuan, paired, category_col_name)
    print('\n--------------------------------------------------------')
    print('--------------------------------------------------------')
    print('tabel asal setelah dipasangkan ke 1')
    print(tabel_asal)
    print('--------------------------------------------------------')
    print('tabel tujuan setelah dipasangkan ke 1')
    print(tabel_tujuan)
    print('--------------------------------------------------------')
    print('hasil yang dipasangkan ke 1')
    print(paired)
    
    new_threshold = len(tabel_asal) * len(tabel_tujuan)
    if threshold == new_threshold and new_threshold != 0 :
        tabel_asal, tabel_tujuan, log_rotasi = process_pairing_2(tabel_asal, tabel_tujuan, paired, category_col_name)
        print('\n\n--------------------------------------------------------')
        print('--------------------------------------------------------')
        print('tabel asal setelah dipasangkan ke 2')
        print(tabel_asal)
        print('--------------------------------------------------------')
        print('tabel tujuan setelah dipasangkan ke 2')
        print(tabel_tujuan)
    threshold = new_threshold

print('--------------------------------------------------------')
print('hasil yang dipasangkan ke 2')
print(log_rotasi)


--------------------------------------------------------
--------------------------------------------------------
tabel asal setelah dipasangkan ke 1
            KOTA          ARTICLE                              STORE ASAL  \
1  KAB. KARAWANG  OPPO A3X 4/64GB    ERAFONE RESINDA PARK KARAWANG (E123)   
2  KAB. KARAWANG  OPPO A3X 4/64GB          ERAFONE RUKO PINAYUNGAN (E641)   
3  KAB. KARAWANG  OPPO A3X 4/64GB      ERAFONE RUKO RENGASDENGKLOK (E409)   
4  KAB. KARAWANG  OPPO A3X 4/64GB   ERAFONE RUKO SUROTOKUNTO KLARI (E940)   
5  KAB. KARAWANG  OPPO A3X 4/64GB               LOTTEMART KARAWANG (G330)   
6  KAB. KARAWANG  OPPO A3X 4/64GB   MEGASTORE RUKO A YANI CIKAMPEK (M064)   
7  KAB. KARAWANG  OPPO A3X 4/64GB  MEGASTORE RUKO A. YANI KARAWANG (M102)   

  TOTAL TEROTASI  
1              2  
2              1  
3              3  
4              1  
5              2  
6              2  
7              2  
--------------------------------------------------------
tabel tujuan setelah di

# Menambahkan informasi stock, sales, dan dos untuk store asal dan tujuan baik sebelum dan sesudah rotasi

In [11]:
def add_dos_information(df_asal, df_asal_updated, df_tujuan, df_tujuan_updated, log_rotasi, category_col_name):
    merged_data = pd.merge(
        log_rotasi, df_asal[['STORE', 'Stock', 'Sales 30 days', 'DOS 30 days']],left_on='STORE ASAL', right_on='STORE', 
        how='left', suffixes=('', '_asal')).drop(columns=['STORE'])  

    merged_data = pd.merge(
        merged_data, df_asal_updated[['STORE', 'Stock', 'Sales 30 days', 'DOS 30 days']], left_on='STORE ASAL', right_on='STORE', 
        how='left',suffixes=('_asal_sebelum', '_asal_setelah')).drop(columns=['STORE'])

    merged_data = pd.merge(
        merged_data, df_tujuan[['STORE', 'Stock', 'Sales 30 days', 'DOS 30 days']], left_on='STORE TUJUAN', right_on='STORE', 
        how='left',suffixes=('', '_tujuan_sebelum')).drop(columns=['STORE'])

    merged_data = pd.merge(
        merged_data, df_tujuan_updated[['STORE', 'Stock', 'Sales 30 days', 'DOS 30 days']], left_on='STORE TUJUAN', right_on='STORE', 
        how='left',suffixes=('_tujuan_sebelum', '_tujuan_setelah')).drop(columns=['STORE'])

    merged_data.rename(columns={
        "Stock_asal_sebelum": "Stock", "Sales 30 days_asal_sebelum": "Sales", "DOS 30 days_asal_sebelum": "Dos",
        "Stock_asal_setelah": "Stock Akhir", "Sales 30 days_asal_setelah": "Sales Akhir", "DOS 30 days_asal_setelah": "Dos Akhir",
        "Stock_tujuan_sebelum": "Stock Tujuan", "Sales 30 days_tujuan_sebelum": "Sales Tujuan", "DOS 30 days_tujuan_sebelum": "Dos Tujuan",
        "Stock_tujuan_setelah": "Stock Akhir Tujuan", "Sales 30 days_tujuan_setelah": "Sales Akhir Tujuan", "DOS 30 days_tujuan_setelah": "Dos Akhir Tujuan"
    }, inplace=True)

    merged_data = merged_data[[
        f"{category_col_name}", "STORE ASAL", "ARTICLE", "Stock", "Sales", "Dos", "Stock Akhir", "Sales Akhir", "Dos Akhir", 
        "TOTAL ASAL", "BARANG TEROTASI", "TOTAL TUJUAN", "STORE TUJUAN", "Stock Tujuan", "Sales Tujuan", "Dos Tujuan", 
        "Stock Akhir Tujuan", "Sales Akhir Tujuan", "Dos Akhir Tujuan"
    ]]

    return merged_data

In [12]:
log_rotasi = add_dos_information(df_asal, df_asal_updated, df_tujuan, df_tujuan_updated, log_rotasi, category_col_name)
print(log_rotasi)

            KOTA                              STORE ASAL          ARTICLE  \
0  KAB. KARAWANG   ERAFONE KARAWANG CENTRAL PLAZA (E054)  OPPO A3X 4/64GB   
1  KAB. KARAWANG    ERAFONE RESINDA PARK KARAWANG (E123)  OPPO A3X 4/64GB   
2  KAB. KARAWANG               LOTTEMART KARAWANG (G330)  OPPO A3X 4/64GB   
3  KAB. KARAWANG      ERAFONE RUKO RENGASDENGKLOK (E409)  OPPO A3X 4/64GB   
4  KAB. KARAWANG   MEGASTORE RUKO A YANI CIKAMPEK (M064)  OPPO A3X 4/64GB   
5  KAB. KARAWANG  MEGASTORE RUKO A. YANI KARAWANG (M102)  OPPO A3X 4/64GB   
6  KAB. KARAWANG          ERAFONE RUKO PINAYUNGAN (E641)  OPPO A3X 4/64GB   
7  KAB. KARAWANG   ERAFONE RUKO SUROTOKUNTO KLARI (E940)  OPPO A3X 4/64GB   

   Stock  Sales   Dos  Stock Akhir  Sales Akhir  Dos Akhir TOTAL ASAL  \
0      4    4.0  30.0            3          4.0       23.0          1   
1      3    0.0   0.0            1          0.0        0.0          2   
2      3    1.0  90.0            1          1.0       30.0          2   
3     21   24.